In [3]:
import json;
import pandas as pd;
import scipy;
from scipy import stats;
# from apyori import apriori;
import matplotlib.pyplot as plt;
import numpy as np;
import sklearn
from sklearn.datasets import load_iris
from sklearn import tree

    
data_path = 'okcupid-profiles-clean-signs.csv'
df = pd.read_csv(data_path)

df = df[df['status'] != 'unknown']
help_dict = {}
help_df = df['sign']
sign_dict = {'aries': 1, 'taurus': 2, 'gemini': 3, 'cancer': 4, 'leo': 5, 'virgo': 6, 'libra': 7, 'scorpio': 8, 'sagittarius': 9, 'capricorn': 10, 'aquarius': 11, 'pisces': 12 }
for i in range(df.shape[0]):
    sign_entry = help_df.iloc[i]
    if sign_entry not in help_dict:
        for sign in sign_dict:
            if sign in sign_entry: 
                help_dict[sign_entry] = sign_dict[sign]

for key in help_dict:
    df.loc[ df['sign'] == key, 'sign'] = help_dict[key]
    for i in range(df.shape[0]):
        if type(df['sign'].iloc[i]) == "<class 'str'>": 
            print(df['sign'].iloc[i])
#print(help_dict)
df = pd.get_dummies(data=df[['age', 'body_type', 'status', 'drugs', 'drinks', 'sign', 'pets', 'height', 'diet']], columns=['body_type', 'status', 'drugs', 'drinks', 'pets', 'diet'])

In [ ]:
''' from sklearn.model_selection import GridSearchCV
params = [{'criterion': ['gini', 'entropy'], 'max_depth': [1, 3, 5, 7, 9, 11, 13, 15, None], 'splitter': ['best', 'random'],
           'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}]
gs_dt = GridSearchCV(decision_tree, param_grid=params, scoring='f1', cv=3, error_score= 'raise')
gs_dt.fit(X_train, y_bool_train)
print(gs_dt.best_params_)

decision_tree_best = tree.DecisionTreeClassifier(**gs_dt.best_params_)
scores2 = sklearn.model_selection.cross_val_score(decision_tree_best, X_train, y_bool_train, cv=5, scoring='f1')
import numpy as np
scores2test = sklearn.model_selection.cross_val_score(decision_tree_best, X_test, y_bool_test, cv=5, scoring='f1')
print(f"Test Dataset CV Average F1 Score: {np.mean(scores2test)}")
print('Test Data Set CV: ', scores2test)
print('Training CV F1 scores: ', scores2)
print('Training CV F1 scores: ', np.mean(scores2))'''

In [4]:
features0 = df.loc[:, df.columns!='sign'].to_numpy()
features, test_data = features0.copy()[0:12680], features0.copy()[12681:]

target0 = df[["sign"]].to_numpy()
target0 = target0.astype('int')

target, goals = target0.copy()[0:12680], target0.copy()[12681:]

from sklearn.model_selection import GridSearchCV
params = [{'criterion': ['gini', 'entropy'], 'max_depth': [1, 3, 5, 7, 9, 11, 13, 15, None], 'splitter': ['best', 'random'],
           'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}]
clf = GridSearchCV(tree.DecisionTreeClassifier(), param_grid=params, cv=3, error_score= 'raise')
clf.fit(features, target)
decision_tree_best = tree.DecisionTreeClassifier(**clf.best_params_)

#plt.figure(figsize=(100,100))
#tree.plot_tree(clf, fontsize=22)
#plt.savefig('tree_high_dpi', dpi=100)


KeyboardInterrupt: 

In [ ]:
decision_tree_best = tree.DecisionTreeClassifier() # **clf.best_params_
decision_tree_best.fit(features, target)
print(type(clf.best_params_))

In [ ]:
from sklearn.ensemble import BaggingClassifier
def accuracy(preds, targets):
    acc = 0
    for i in range(preds.shape[0]):
        if preds[i] == targets[i]:
            acc +=1
    return acc/preds.shape[0]
sil = tree.DecisionTreeClassifier().fit(features, target)
bgClssfr = BaggingClassifier(base_estimator=sil, n_estimators=100000)
predictions = sil.predict(test_data)
#predictions = clf.predict(test_data)

estimators = [5, 10, 100, 1000, 10000, 100000]
for j in estimators:
    bc = BaggingClassifier(base_estimator=sil, n_estimators=j)
    bc.fit(features, target)
    avg_pred = []
    for i in range(1):
        predictions = bc.predict(test_data)
        avg_pred.append(accuracy(predictions, goals))
    print(np.mean(avg_pred))

In [ ]:
df